# Action Provider Demo: send requests to Globus Flows
Adapted from https://github.com/globus/globus-jupyter-notebooks/blob/master/Automation_Using_Globus_Flows.ipynb

## 1. Setup

### 1.1 Log in with Globus Auth identity

In [ ]:
"""Use Diaspora AP in Globus Flows."""

from __future__ import annotations

import json
import time
import uuid

import globus_sdk
import globus_sdk.scopes
from diaspora_event_sdk import Client as GlobusClient

# ID of this tutorial notebook as registered with Globus Auth
CLIENT_ID = 'f794186b-f330-4595-b6c6-9c9d3e903e47'

# Do a native app authentication flow to get tokens that allow us
# to interact with the Globus Flows service

scopes = [
    'openid',
    'profile',
    'email',
    globus_sdk.FlowsClient.scopes.manage_flows,
    globus_sdk.FlowsClient.scopes.run_manage,
]
native_auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
native_auth_client.oauth2_start_flow(requested_scopes=scopes)
print(f'Login Here:\n\n{native_auth_client.oauth2_get_authorize_url()}')

auth_code = input('Authorization Code: ')
response = native_auth_client.oauth2_exchange_code_for_tokens(auth_code)

tokens = response.by_resource_server
print(json.dumps(tokens, indent=2))

flows_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens['flows.globus.org']['access_token'],
)
flows_client = globus_sdk.FlowsClient(authorizer=flows_authorizer)

# Create an Auth client so we can look up identities
auth_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens['auth.globus.org']['access_token'],
)
ac = globus_sdk.AuthClient(authorizer=auth_authorizer)

# Get the user's primary identity
primary_identity = ac.oauth2_userinfo()
identity_id = primary_identity['sub']

print(f'Username: {primary_identity["preferred_username"]}')
print(f'ID: {identity_id}')

### 1.2 Select a Topic

In [ ]:
c = GlobusClient()
print("User's OpenID:", c.subject_openid)
topic = 'topic-' + c.subject_openid[-12:]
print(c.register_topic(topic))
print(c.list_topics())
print('Topic to produce/consume:', topic)

### 1.3 Helper functions

In [ ]:
action_url = 'https://diaspora-action-provider.ml22sevubfnks.us-east-1.cs.amazonlightsail.com/'


def run_flow(specific_flow_client, flow_input):  # noqa: D103
    run_label = f'Diaspora AP Flow by {primary_identity["preferred_username"]}'
    run = specific_flow_client.run_flow(
        body=flow_input,
        label=run_label,
        tags=['tutorial', 'diaspora'],
    )

    # Get run details
    run_id = run['run_id']
    run_status = run['status']
    print('This flow can be monitored in the Web App:')
    print(f'https://app.globus.org/runs/{run_id}')
    print(f'Flow run started with ID: {run_id} - Status: {run_status}')

    # Poll the Flow service to check on the status of the flow
    counter, max_count = 0, 6
    while run_status == 'ACTIVE' and counter < max_count:
        time.sleep(5)
        run = flows_client.get_run(run_id)
        run_status = run['status']
        print(f'Run status: {run_status}')
        counter += 1

    # Run completed
    # print(json.dumps(run.data, indent=2))
    print(run.data)
    return run.data

## 2.1 Produce a single message to AP with `key` and `value`

### 2.1.1 Produce a single message to AP without `key`

In [ ]:
flow_definition211 = {
    'Comment': 'Publish messages to Diaspora Event Fabric',
    'StartAt': 'PublishMessages',
    'States': {
        'PublishMessages': {
            'Comment': 'Send messages to a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
                'value.$': '$.input.value',
            },
            'ResultPath': '$.PublishMessages',
            'End': True,
        },
    },
}

In [ ]:
flow_title = f'Diapora-AP-Flow-{str(uuid.uuid4())[:4]}'
flow = flows_client.create_flow(
    title=flow_title,
    definition=flow_definition211,
    input_schema={},
)
flow_id = flow['id']
flow_scope = globus_sdk.SpecificFlowClient(flow_id).scopes.make_mutable('user')
print(f"Successfully created flow: '{flow_title} (ID: {flow_id})")
print(f'View the flow in the Web App: https://app.globus.org/flows/{flow_id}')

if flow_id not in tokens:
    # Do a native app authentication flow and get tokens that
    # include the newly deployed flow scope
    native_auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
    native_auth_client.oauth2_start_flow(requested_scopes=flow_scope)
    print(f'Login Here:\n\n{native_auth_client.oauth2_get_authorize_url()}')

    # Authenticate and come back with your authorization code;
    # paste it into the prompt below.
    auth_code = input('Authorization Code: ')
    token_response = native_auth_client.oauth2_exchange_code_for_tokens(
        auth_code,
    )

    # Save the new token in a place where the flows client can retrieve it.
    tokens[flow_id] = token_response.by_resource_server[flow_id]

    # These are the saved scopes for the flow
    # print(json.dumps(tokens, indent=2))

# Get a client for the flow
specific_flow_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens[flow_id]['access_token'],
)
# print(tokens[flow_id]['access_token'])
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    authorizer=specific_flow_authorizer,
)

In [ ]:
flow_input211 = {
    'input': {
        'action': 'produce',
        'topic': topic,
        'value': {'content': 'hello world1'},
    },
}
run_data = run_flow(specific_flow_client, flow_input211)
assert run_data['status'] == 'SUCCEEDED'

### 2.1.2 Produce a single message to AP with `key`

In [ ]:
flow_definition212 = {
    'Comment': 'Publish messages to Diaspora Event Fabric',
    'StartAt': 'PublishMessages',
    'States': {
        'PublishMessages': {
            'Comment': 'Send messages to a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
                'key.$': '$.input.key',
                'value.$': '$.input.value',
            },
            'ResultPath': '$.PublishMessages',
            'End': True,
        },
    },
}
flows_client.update_flow(
    flow_id,
    definition=flow_definition212,
    input_schema={},
)

In [ ]:
flow_input212 = {
    'input': {
        'action': 'produce',
        'topic': topic,
        'key': 'my-key-123',
        'value': {'content': 'hello world1'},
    },
}
run_data = run_flow(specific_flow_client, flow_input212)
assert run_data['status'] == 'SUCCEEDED'

## 2.2 Produce a batch of messages to AP with `keys` and `msgs`

### 2.2.1 Produce messages to AP without keys

In [ ]:
flow_definition221 = {
    'Comment': 'Publish messages to Diaspora Event Fabric',
    'StartAt': 'PublishMessages',
    'States': {
        'PublishMessages': {
            'Comment': 'Send messages to a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
                'msgs.$': '$.input.msgs',
            },
            'ResultPath': '$.PublishMessages',
            'End': True,
        },
    },
}
flows_client.update_flow(
    flow_id,
    definition=flow_definition221,
    input_schema={},
)

In [ ]:
flow_input221 = {
    'input': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}
run_data = run_flow(specific_flow_client, flow_input221)
assert run_data['status'] == 'SUCCEEDED'

### 2.2.2 Produce messages to AP with a key

In [ ]:
flow_definition222 = {
    'Comment': 'Publish messages to Diaspora Event Fabric',
    'StartAt': 'PublishMessages',
    'States': {
        'PublishMessages': {
            'Comment': 'Send messages to a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
                'msgs.$': '$.input.msgs',
                'keys.$': '$.input.keys',
            },
            'ResultPath': '$.PublishMessages',
            'End': True,
        },
    },
}
flows_client.update_flow(
    flow_id,
    definition=flow_definition222,
    input_schema={},
)

In [ ]:
flow_input222 = {
    'input': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
        'keys': 'my-key-123',
    },
}
run_data = run_flow(specific_flow_client, flow_input222)
assert run_data['status'] == 'SUCCEEDED'

### 2.2.3 Produce messages to AP with a list of keys

In [ ]:
flow_input223 = {
    'input': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
        'keys': [
            'my-key-123',
            'my-key-456',
            'my-key-789',
        ],
    },
}
run_data = run_flow(specific_flow_client, flow_input223)
assert run_data['status'] == 'SUCCEEDED'

## 3.1 Blocking consume returns `ACTIVE`, produce more events to unblock it

In [ ]:
one_hour_ago = (int(time.time()) - 3600) * 1000

### 3.1.1 Consume messages within one hour (with `ts`, with `group_id`)

In [ ]:
flow_definition311 = {
    'Comment': 'Consume messages to Diaspora Event Fabric',
    'StartAt': 'ConsumeMessages',
    'States': {
        'ConsumeMessages': {
            'Comment': 'Receive messages from a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
                'ts.$': '$.input.ts',
                'group_id.$': '$.input.group_id',
            },
            'ResultPath': '$.ConsumeMessages',
            'End': True,
        },
    },
}

flows_client.update_flow(
    flow_id,
    definition=flow_definition311,
    input_schema={},
)

In [ ]:
flow_input311 = {
    'input': {
        'action': 'consume',
        'topic': topic,
        'ts': one_hour_ago,
        'group_id': 'my-group-1234',
    },
}
run_data = run_flow(specific_flow_client, flow_input311)
assert run_data['status'] == 'SUCCEEDED'

### 3.1.2 The same request body returns `ACTIVE` status with no message

In [ ]:
run_data = run_flow(specific_flow_client, flow_input311)
assert run_data['status'] == 'ACTIVE'
consume_run_id = run_data['run_id']

### 3.1.3 Produce a few more messages

In [ ]:
flow_definition313 = {
    'Comment': 'Publish messages to Diaspora Event Fabric',
    'StartAt': 'PublishMessages',
    'States': {
        'PublishMessages': {
            'Comment': 'Send messages to a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
                'msgs.$': '$.input.msgs',
            },
            'ResultPath': '$.PublishMessages',
            'End': True,
        },
    },
}
flows_client.update_flow(
    flow_id,
    definition=flow_definition313,
    input_schema={},
)

In [ ]:
flow_input313 = {
    'input': {
        'action': 'produce',
        'topic': topic,
        'msgs': [
            {'content': 'hello world1'},
            {'content': 'hello world2'},
            {'content': 'hello world3'},
        ],
    },
}
run_data = run_flow(specific_flow_client, flow_input313)
assert run_data['status'] == 'SUCCEEDED'

### 3.1.4 Inspect the consumer request, which should be finished shortly.

In [ ]:
run_data = flows_client.get_run(consume_run_id)
print(run_data)
assert run_data['status'] == 'SUCCEEDED'

### 3.1.5 Unfinished runs can be cancelled, and finished runs can be released

In [ ]:
# TODO

## 3.2 Consume with filters

`filters` specifies a list of filters, an event must meet all conditions of at least one filter for it to be returned.

`filters` can be used with `ts` or `group_id` or both.

In [ ]:
flow_definition321 = {
    'Comment': 'Consume messages to Diaspora Event Fabric',
    'StartAt': 'ConsumeMessages',
    'States': {
        'ConsumeMessages': {
            'Comment': 'Receive messages from a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
                'filters.$': '$.input.filters',
            },
            'ResultPath': '$.ConsumeMessages',
            'End': True,
        },
    },
}

flows_client.update_flow(
    flow_id,
    definition=flow_definition321,
    input_schema={},
)

Here, an event is returned if its `content` meets one of the two filters:

filter 1: events with prefix `hello world1`

filter 2: events with prefix `hello` AND suffix `world2`

In [ ]:
flow_input321 = {
    'input': {
        'action': 'consume',
        'topic': topic,
        'filters': [
            {'Pattern': {'value': {'content': [{'prefix': 'hello world1'}]}}},
            {
                'Pattern': {
                    'value': {
                        'content': [
                            {'prefix': 'hello', 'suffix': 'world2'},
                        ],
                    },
                },
            },
        ],
    },
}
run_data = run_flow(specific_flow_client, flow_input321)
assert run_data['status'] == 'SUCCEEDED'